# Time Series Data Aggregation and Feature Extraction

This script processes multiple datasets for various utilities (e.g., chilled water, electricity, gas), merging time series data for each and performing daily resampling and aggregation. After merging, additional features are calculated, and train/test datasets are generated for modeling or further analysis.

## Overview

The script performs the following tasks for each dataset:

1. **Merge Data**: Combines consumption data with weather data.
2. **Resample and Aggregate**: Resamples to daily frequency, calculates daily aggregates, and extracts time-related features.
3. **Split into Train and Test Sets**: Splits the data into 2016 (train) and 2017 (test) datasets.

## Code Details

### Global Variables

- `DATASET_NAMES`: List of utility datasets to process, such as "chilledwater" and "electricity".
- `data_map`: Dictionary containing the site, building, and consumer configurations for each dataset type.
- `TARGET_FILE_TEMPLATE`: Template path for the meter data file for each dataset.
- `WEATHER_FILE`: Path for the weather data file.
- `OUTPUT_DIR`: Directory where the final processed files will be saved.

### Functions

#### 1. `extract_and_merge_data(dataset_name, config)`

This function merges the meter and weather data, performs daily resampling, calculates aggregation-based features, and saves the results.

- **Parameters**:

  - `dataset_name` (str): Name of the dataset (e.g., "water").
  - `config` (dict): Configuration containing `site_name`, `building_name`, and `consumer_name`.

- **Process**:

  1. **File Paths and Columns**: Constructs paths and target columns based on dataset configuration.
     - Output paths are organized in subfolders using `dataset_name`, `site_name`, `building_name`, and `consumer_name`, so each file has a dedicated folder.
  2. **Data Loading**: Reads the meter and weather data, ensuring only relevant columns are loaded.
  3. **Merging**: Merges data on `timestamp` and keeps only relevant timestamps.
  4. **Daily Resampling**:
     - Resamples on daily frequency (`'D'`) using `sum` for the primary target column and `mean` for weather data.
     - Calculates the following features for each day:
       - `sum_conso`: Daily sum of consumption.
       - `mean_conso`: Daily mean of consumption.
       - `min_conso`: Daily minimum consumption.
       - `max_conso`: Daily maximum consumption.
       - `first_conso`: First recorded consumption value for the day.
       - `last_conso`: Last recorded consumption value for the day.
       - `median_conso`: Median consumption for the day.
       - `month`: Month extracted from `timestamp`.
       - `day_of_week`: Day of the week (Monday = 0, Sunday = 6).
  5. **Remove Timestamp Column**: After resampling and feature generation, the `timestamp` column is removed from the final processed file.
  6. **Error Handling**: Catches and logs any issues with file reading, column mismatches, or merging issues.

- **Returns**:
  - `output_file` (str): Path to the saved file with the resampled and aggregated data, or `None` if an error occurs.

#### 2. `split_train_test(output_file)`

This function splits the resampled dataset into train (2016) and test (2017) datasets.

- **Parameters**:

  - `output_file` (str): Path to the file generated by `extract_and_merge_data`.

- **Process**:

  1. **Load Data**: Reads the `output_file`.
  2. **Train/Test Split**:
     - Filters rows with `timestamp` in 2016 as the train set.
     - Filters rows with `timestamp` in 2017 as the test set.
  3. **Save Train and Test Files**:
     - Saves the train set as `_TRAIN.CSV` and the test set as `_TEST.CSV` in their respective folders.

- **Error Handling**: Catches any issues during the loading or filtering process and logs an error.

### Main Execution Loop

Iterates over each dataset in `DATASET_NAMES` and each configuration in `data_map`:

1. Calls `extract_and_merge_data` to process and save the aggregated data.
2. Calls `split_train_test` to split the aggregated data into train and test sets.

### Example Usage

```python
# Example to process each dataset and configuration
for dataset_name in DATASET_NAMES:
    if dataset_name in data_map:
        for index, config in data_map[dataset_name].items():
            output_file = extract_and_merge_data(dataset_name, config)
            split_train_test(output_file)
```


In [1]:
import pandas as pd
import os

# Define dataset names and data map with configurations for each dataset type
DATASET_NAMES = [
    "electricity",
    "gas",
    "hotwater",
    "solar",
    "water",
]
data_map = {
    "electricity": {
        0: {
            "site_name": "Mouse",
            "building_name": "science",
            "consumer_name": "Micheal",
        },
        1: {"site_name": "Mouse", "building_name": "health", "consumer_name": "Estela"},
    },
    "gas": {
        0: {
            "site_name": "Panther",
            "building_name": "education",
            "consumer_name": "Mohammad",
        },
        1: {
            "site_name": "Panther",
            "building_name": "lodging",
            "consumer_name": "Dean",
        },
    },
    "hotwater": {
        0: {"site_name": "Fox", "building_name": "lodging", "consumer_name": "Alana"},
        1: {
            "site_name": "Robin",
            "building_name": "education",
            "consumer_name": "Margarito",
        },
    },
    "solar": {
        0: {
            "site_name": "Bobcat",
            "building_name": "education",
            "consumer_name": "Alissa",
        },
        1: {
            "site_name": "Bobcat",
            "building_name": "education",
            "consumer_name": "Coleman",
        },
    },
    "water": {
        0: {
            "site_name": "Panther",
            "building_name": "lodging",
            "consumer_name": "Cora",
        },
        1: {
            "site_name": "Wolf",
            "building_name": "education",
            "consumer_name": "Ursula",
        },
    },
}

TARGET_FILE_TEMPLATE = "../data/meters/cleaned/{}_cleaned.csv"
WEATHER_FILE = "../data/weather/weather.csv"
OUTPUT_DIR = "../data/meters/final/"


def extract_and_merge_data(dataset_name, config):
    site_name = config["site_name"]
    building_name = config["building_name"]
    consumer_name = config["consumer_name"]

    # Define file paths and target/output columns
    target_file = TARGET_FILE_TEMPLATE.format(dataset_name)
    processed_file = f"{dataset_name}_{site_name}_{building_name}_{consumer_name}"
    output_file = os.path.join(
        OUTPUT_DIR,
        processed_file.upper(),
        f"{processed_file}.csv".upper(),
    )

    target_column = f"{site_name}_{building_name}_{consumer_name}"
    new_target_column = dataset_name.capitalize()

    try:
        # Load target data (meter readings)
        target_df = pd.read_csv(target_file, usecols=["timestamp", target_column])
        target_df = target_df.rename(columns={target_column: new_target_column})
        target_df["timestamp"] = pd.to_datetime(target_df["timestamp"])

        # Load weather data, filter for the specific site, and drop `site_id`
        weather_df = pd.read_csv(WEATHER_FILE)
        weather_df = weather_df[weather_df["site_id"] == site_name].drop(
            columns=["site_id"]
        )
        weather_df["timestamp"] = pd.to_datetime(weather_df["timestamp"])

        # Merge on timestamp, keeping timestamps from the target data only
        merged_df = pd.merge(target_df, weather_df, on="timestamp", how="left")

        # Resample the data to daily frequency using sum for consumption and mean for weather data
        resampled_df = merged_df.resample("D", on="timestamp").agg(
            {
                new_target_column: [
                    "sum",
                    "mean",
                    "min",
                    "max",
                    "first",
                    "last",
                    "median",
                ],
                **{col: "mean" for col in weather_df.columns if col != "timestamp"},
            }
        )

        # Flatten column names after aggregation
        resampled_df.columns = [
            "sum_conso",
            "mean_conso",
            "min_conso",
            "max_conso",
            "first_conso",
            "last_conso",
            "median_conso",
        ] + list(weather_df.columns[1:])
        resampled_df.reset_index(inplace=True)

        # Add time-related features
        resampled_df["month"] = resampled_df["timestamp"].dt.month
        resampled_df["day_of_week"] = resampled_df["timestamp"].dt.dayofweek

        # Save to the specified output file without removing `timestamp`
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        resampled_df.to_csv(output_file, index=False)

        print(f"Data extracted and saved to {output_file}")
        return output_file

    except FileNotFoundError as e:
        print(
            f"File not found: {e}. Skipping {dataset_name} for {site_name} - {building_name} - {consumer_name}."
        )
    except ValueError as e:
        print(
            f"Column error: {e}. Skipping {dataset_name} for {site_name} - {building_name} - {consumer_name}."
        )
    except Exception as e:
        print(
            f"An error occurred: {e}. Skipping {dataset_name} for {site_name} - {building_name} - {consumer_name}."
        )

    return None


def split_train_test(output_file):
    if output_file is None:
        return

    try:
        # Load the merged data with `timestamp` included
        merged_df = pd.read_csv(output_file, parse_dates=["timestamp"])

        # Filter rows by year for train and test sets
        train_df = merged_df[merged_df["timestamp"].dt.year == 2016]
        test_df = merged_df[merged_df["timestamp"].dt.year == 2017]

        # Define subfolder paths for train and test data
        train_file = output_file.replace(".CSV", "_TRAIN.CSV")
        test_file = output_file.replace(".CSV", "_TEST.CSV")

        # Drop `timestamp` in final train and test outputs
        train_df = train_df.drop(columns=["timestamp"])
        test_df = test_df.drop(columns=["timestamp"])

        # Save train and test dataframes to their respective files
        os.makedirs(os.path.dirname(train_file), exist_ok=True)
        train_df.to_csv(train_file, index=False)
        test_df.to_csv(test_file, index=False)

        print(f"Train data saved to {train_file}")
        print(f"Test data saved to {test_file}")

    except Exception as e:
        print(
            f"An error occurred while splitting data: {e}. Skipping file {output_file}."
        )


# Process each dataset in DATASET_NAMES and each configuration in data_map
for dataset_name in DATASET_NAMES:
    if dataset_name in data_map:
        for index, config in data_map[dataset_name].items():
            output_file = extract_and_merge_data(dataset_name, config)
            split_train_test(output_file)


Data extracted and saved to ../data/meters/final/ELECTRICITY_MOUSE_SCIENCE_MICHEAL/ELECTRICITY_MOUSE_SCIENCE_MICHEAL.CSV
Train data saved to ../data/meters/final/ELECTRICITY_MOUSE_SCIENCE_MICHEAL/ELECTRICITY_MOUSE_SCIENCE_MICHEAL_TRAIN.CSV
Test data saved to ../data/meters/final/ELECTRICITY_MOUSE_SCIENCE_MICHEAL/ELECTRICITY_MOUSE_SCIENCE_MICHEAL_TEST.CSV
Data extracted and saved to ../data/meters/final/ELECTRICITY_MOUSE_HEALTH_ESTELA/ELECTRICITY_MOUSE_HEALTH_ESTELA.CSV
Train data saved to ../data/meters/final/ELECTRICITY_MOUSE_HEALTH_ESTELA/ELECTRICITY_MOUSE_HEALTH_ESTELA_TRAIN.CSV
Test data saved to ../data/meters/final/ELECTRICITY_MOUSE_HEALTH_ESTELA/ELECTRICITY_MOUSE_HEALTH_ESTELA_TEST.CSV
Data extracted and saved to ../data/meters/final/GAS_PANTHER_EDUCATION_MOHAMMAD/GAS_PANTHER_EDUCATION_MOHAMMAD.CSV
Train data saved to ../data/meters/final/GAS_PANTHER_EDUCATION_MOHAMMAD/GAS_PANTHER_EDUCATION_MOHAMMAD_TRAIN.CSV
Test data saved to ../data/meters/final/GAS_PANTHER_EDUCATION_MOHAMMAD

In [2]:
import pandas as pd
import os

# Define paths and constants
TARGET_FILE_ELECTRICITY = "../data/ELECTRICITY/ELECTRICITY.txt"
OUTPUT_DIR = "../data/meters/final/ELECTRICITY/"


def process_electricity_data():
    # Define file path and output file name
    processed_file = "electricity_global_reactive_power"
    output_file = os.path.join(
        OUTPUT_DIR, f"{processed_file}.csv".upper()
    )

    # Load the electricity data with Date and Time combined into a timestamp
    try:
        electricity_df = pd.read_csv(
            TARGET_FILE_ELECTRICITY,
            sep=";",  # Separator is ';'
            parse_dates=[[0, 1]],  # Combine 'Date' and 'Time' columns
            dayfirst=True,  # Use day-first format for dates
            na_values="?",  # Handle missing values
        )
        electricity_df.columns = [
            "timestamp",
            "Global_active_power",
            "Global_reactive_power",
            "Voltage",
            "Global_intensity",
            "Sub_metering_1",
            "Sub_metering_2",
            "Sub_metering_3",
        ]
        electricity_df["timestamp"] = pd.to_datetime(
            electricity_df["timestamp"], errors="coerce"
        )

        # Drop rows with invalid dates
        electricity_df.dropna(subset=["timestamp"], inplace=True)

        # Define detailed aggregations for `Global_reactive_power`
        aggregation_dict = {
            "Global_reactive_power": [
                "sum",
                "mean",
                "min",
                "max",
                "first",
                "last",
                "median",
            ]  # Detailed aggregations
        }

        # Simple sum aggregation for other columns
        other_columns = {
            "Voltage": "sum",
            "Global_intensity": "sum",
            "Sub_metering_1": "sum",
            "Sub_metering_2": "sum",
            "Sub_metering_3": "sum",
        }

        # Combine all aggregations into a single dictionary
        aggregation_dict.update(other_columns)

        # Resample the data to daily frequency using the specified aggregations
        resampled_df = electricity_df.resample("D", on="timestamp").agg(
            aggregation_dict
        )

        # Rename columns for `Global_reactive_power` aggregations only
        resampled_df.columns = [
            "sum_conso",
            "mean_conso",
            "min_conso",
            "max_conso",
            "first_conso",
            "last_conso",
            "median_conso",
        ] + list(other_columns.keys())

        resampled_df.reset_index(inplace=True)

        # Add time-related features
        resampled_df["month"] = resampled_df["timestamp"].dt.month
        resampled_df["day_of_week"] = resampled_df["timestamp"].dt.dayofweek

        # Convert all column names to lowercase
        resampled_df.columns = [col.lower() for col in resampled_df.columns]

        # Save to the specified output file
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        resampled_df.to_csv(output_file, index=False)

        print(f"Electricity data processed and saved to {output_file}")
        return output_file

    except FileNotFoundError as e:
        print(f"File not found: {e}. Skipping electricity data.")
    except ValueError as e:
        print(f"Value error: {e}. Skipping electricity data.")
    except Exception as e:
        print(f"An error occurred: {e}. Skipping electricity data.")

    return None


def split_train_test(output_file):
    if output_file is None:
        return

    try:
        # Load the processed data with `timestamp` included
        merged_df = pd.read_csv(output_file, parse_dates=["timestamp"])

        # Filter rows by year for train (2006–2008) and test (2009–2010) sets
        train_df = merged_df[merged_df["timestamp"].dt.year.isin([2006, 2007, 2008])]
        test_df = merged_df[merged_df["timestamp"].dt.year.isin([2009, 2010])]

        # Define subfolder paths for train and test data
        train_file = output_file.replace(".CSV", "_train.csv")
        test_file = output_file.replace(".CSV", "_test.csv")

        # Drop `timestamp` before saving the final train and test files
        train_df.drop(columns=["timestamp"], inplace=True)
        test_df.drop(columns=["timestamp"], inplace=True)

        # Save train and test dataframes to their respective files
        os.makedirs(os.path.dirname(train_file), exist_ok=True)
        train_df.to_csv(train_file, index=False)
        test_df.to_csv(test_file, index=False)

        print(f"Train data saved to {train_file}")
        print(f"Test data saved to {test_file}")

    except Exception as e:
        print(
            f"An error occurred while splitting data: {e}. Skipping file {output_file}."
        )


# Run the process for ELECTRICITY.txt
output_file = process_electricity_data()
split_train_test(output_file)


/var/folders/b8/20d44f8s2m19rh4cjlzb01lc0000gn/T/ipykernel_22180/2736359685.py:18: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  electricity_df = pd.read_csv(


Electricity data processed and saved to ../data/meters/final/ELECTRICITY/ELECTRICITY_GLOBAL_REACTIVE_POWER.CSV
Train data saved to ../data/meters/final/ELECTRICITY/ELECTRICITY_GLOBAL_REACTIVE_POWER_train.csv
Test data saved to ../data/meters/final/ELECTRICITY/ELECTRICITY_GLOBAL_REACTIVE_POWER_test.csv


/var/folders/b8/20d44f8s2m19rh4cjlzb01lc0000gn/T/ipykernel_22180/2736359685.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(columns=["timestamp"], inplace=True)
/var/folders/b8/20d44f8s2m19rh4cjlzb01lc0000gn/T/ipykernel_22180/2736359685.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(columns=["timestamp"], inplace=True)


Done!

In [6]:
import pandas as pd

# Path to the CSV file
file_path = "../data/meters/final/ELECTRICITY_MOUSE_HEALTH_ESTELA/ELECTRICITY_MOUSE_HEALTH_ESTELA_TRAIN.CSV"

try:
    # Load the CSV file into a DataFrame
    data = pd.read_csv(file_path)
    
    # Display a summary of the DataFrame
    print("Summary of the dataset:")
    print(data.describe(include='all'))  # Summary statistics for each column

    # Additional summary information (optional)
    print("\nAdditional Information:")
    print(f"Number of rows: {data.shape[0]}")
    print(f"Number of columns: {data.shape[1]}")
    print("\nColumn Types:")
    print(data.dtypes)
    
except FileNotFoundError:
    print(f"Error: File '{file_path}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Summary of the dataset:
          sum_conso  mean_conso   min_conso   max_conso  first_conso  \
count    366.000000  366.000000  366.000000  366.000000   366.000000   
mean    9263.622917  385.984288  301.911258  488.600001   316.520767   
std      998.292054   41.595502   32.141453   89.564894    27.813451   
min     6971.000000  290.458333   74.000000  313.000000   135.000000   
25%     8293.500000  345.562500  284.000000  391.000000   300.000000   
50%     9547.081250  397.795052  296.000000  537.000000   312.000000   
75%     9964.000000  415.166667  311.000000  554.000000   325.000000   
max    11315.000000  471.458333  465.567000  662.000000   465.567000   

       last_conso  median_conso  airTemperature  cloudCoverage  \
count  366.000000    366.000000      366.000000     245.000000   
mean   325.056285    362.595495       11.697981       0.119125   
std     28.395152     31.356386        5.595295       0.897689   
min    274.000000    287.500000        0.175000       0.000000 

In [7]:
data.head()

,sum_conso,mean_conso,min_conso,max_conso,first_conso,last_conso,median_conso,airTemperature,cloudCoverage,dewTemperature,precipDepth1HR,precipDepth6HR,seaLvlPressure,windDirection,windSpeed,month,day_of_week
0,6971.0,290.458333,135.0,326.0,135.0,282.0,298.0,5.387500,0.0,3.879167,NaN,0.0,1016.941667,116.666667,4.470833,1,4
1,7257.0,302.375000,282.0,327.0,288.0,301.0,304.0,9.783333,NaN,9.183333,NaN,15.0,998.087500,166.666667,7.241667,1,5
2,7435.0,309.791667,296.0,325.0,296.0,302.0,311.5,7.954167,0.0,6.687500,NaN,6.0,991.837500,187.916667,5.633333,1,6
3,9074.0,378.083333,278.0,501.0,294.0,302.0,344.0,7.841667,0.0,6.508333,NaN,0.0,982.766667,177.083333,4.145833,1,0
4,9514.0,396.416667,288.0,542.0,293.0,301.0,355.5,7.904167,NaN,7.058333,NaN,0.5,984.712500,175.416667,2.750000,1,1
